In [1]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku

from google.colab import files
uploaded = files.upload()
from data import text_data_arr

# Your text data
data = text_data_arr

tokenizer = Tokenizer(char_level=True)

tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1

# convert data to sequence of tokens
input_sequences = []
output_sequences = []

for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence[:-1])
        output_sequences.append(n_gram_sequence[-1])

# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
model.fit(predictors, label, epochs=50, verbose=1)

Saving data.py to data.py
Epoch 1/50
787/787 [==============================] - 108s 125ms/step - loss: 2.9026 - accuracy: 0.2099
Epoch 2/50
787/787 [==============================] - 97s 123ms/step - loss: 2.5894 - accuracy: 0.2738
Epoch 3/50
787/787 [==============================] - 97s 124ms/step - loss: 2.5253 - accuracy: 0.2840
Epoch 4/50
787/787 [==============================] - 97s 123ms/step - loss: 2.2645 - accuracy: 0.3422
Epoch 5/50
787/787 [==============================] - 98s 124ms/step - loss: 2.0996 - accuracy: 0.3901
Epoch 6/50
787/787 [==============================] - 97s 124ms/step - loss: 1.9476 - accuracy: 0.4363
Epoch 7/50
787/787 [==============================] - 97s 123ms/step - loss: 1.7785 - accuracy: 0.4872
Epoch 8/50
787/787 [==============================] - 97s 124ms/step - loss: 1.6218 - accuracy: 0.5343
Epoch 9/50
787/787 [==============================] - 97s 124ms/step - loss: 1.4798 - accuracy: 0.5755
Epoch 10/50
787/787 [=========================

In [2]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0).flatten()
        predicted = np.random.choice(range(total_words), p=predicted_probs)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += output_word
    return seed_text

print(generate_text("John: How are you, Sarah?", 100, model, max_sequence_len))

John: How are you, Sarah? thanks for the rempens. some of them weake it going to be tryen. they have some really good dlaning
